In [33]:
import pandas as pd 
import numpy as np
import spacy

In [34]:
df=pd.read_csv("C:/Users/nr802/Downloads/GenAI Projects/twitter_training.csv")

In [35]:
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   2401                                                   74681 non-null  int64 
 1   Borderlands                                            74681 non-null  object
 2   Positive                                               74681 non-null  object
 3   im getting on borderlands and i will murder you all ,  73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [5]:
df.isna().sum()

2401                                                       0
Borderlands                                                0
Positive                                                   0
im getting on borderlands and i will murder you all ,    686
dtype: int64

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.isna().sum()

2401                                                       0
Borderlands                                                0
Positive                                                   0
im getting on borderlands and i will murder you all ,    326
dtype: int64

In [8]:
df.shape

(71981, 4)

In [9]:
df.dropna(inplace=True)

In [10]:
df.shape

(71655, 4)

In [11]:
df.isna().sum()

2401                                                     0
Borderlands                                              0
Positive                                                 0
im getting on borderlands and i will murder you all ,    0
dtype: int64

In [12]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------- -------------------------------- 2.4/12.8 MB 13.4 MB/s eta 0:00:01
     --------------------- ------------------ 6.8/12.8 MB 18.3 MB/s eta 0:00:01
     ------------------------------- ------- 10.5/12.8 MB 20.4 MB/s eta 0:00:01
     ------------------------------------- - 12.3/12.8 MB 16.1 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 15.5 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 15.5 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 15.5 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 15.5 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 15.5 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 15.5 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 15.5 MB/s eta 0:00:01
     --------------------------------------  12.

In [12]:
nlp = spacy.load("en_core_web_sm")

In [14]:
import re

# Define a sample preprocessing function
def preprocess(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply the preprocessing function to the 'Text' column
df['Preprocessed Text'] = df['Text'].apply(preprocess)


In [16]:
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

In [19]:
df.columns

Index(['ID', 'Category', 'Sentiment', 'Text', 'Preprocessed Text'], dtype='object')

In [22]:
from sklearn.preprocessing import LabelEncoder


# Encode the 'Sentiment' column
le_model = LabelEncoder()
df['label'] = le_model.fit_transform(df['Sentiment'])

In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
le_model = LabelEncoder()
df['label'] = le_model.fit_transform(df['label'])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df['Preprocessed Text'], df['label'],test_size=0.2, random_state=42, stratify=df['label'])

In [25]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (57324,)
Shape of X_test:  (14331,)


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)
y_pred = model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.63


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [30]:
rf = RandomForestClassifier()
rf.fit(X_train_vectorized, y_train)

RandomForestClassifier()

In [31]:
y_pred_rf = rf.predict(X_test_vectorized)

In [32]:
print("Accuracy:", accuracy_score(y_test, y_pred_rf))

Accuracy: 0.8797013467308632
